# Note that Everything in this notebook until the markdown cell that says "Part 1: Training RNN With Varying Hyperparameters (Altering Hidden Layer Sizes Etc.)" is repeated from the other notebook and doesn't need close attention. 

# However after this part 1 cell is where I have tried different hyperparameter combinations to get a converging RNN model for the dataset.  At the end of this notebook I successfully save my best models weights to be used in the other jupyter notebook without training a model!

In [1]:
import torch
import torch.nn.functional as F
import torchtext
import time
import random
import pandas as pd

torch.backends.cudnn.deterministic = True

In [2]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 0.005
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 16        #128
HIDDEN_DIM =  24         #256
NUM_CLASSES = 2

In [3]:
!wget https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz

--2022-01-26 00:09:41--  https://github.com/rasbt/python-machine-learning-book-3rd-edition/raw/master/ch08/movie_data.csv.gz
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz [following]
--2022-01-26 00:09:42--  https://raw.githubusercontent.com/rasbt/python-machine-learning-book-3rd-edition/master/ch08/movie_data.csv.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26521894 (25M) [application/octet-stream]
Saving to: ‘movie_data.csv.gz’

movie_data.csv.gz   100%[===================>]  25.29M  13.8MB/s  

In [4]:
!gunzip -f movie_data.csv.gz 

In [5]:
df = pd.read_csv('movie_data.csv')
print(df)
df.head()

                                                  review  sentiment
0      In 1974, the teenager Martha Moxley (Maggie Gr...          1
1      OK... so... I really like Kris Kristofferson a...          0
2      ***SPOILER*** Do not read this, if you think a...          0
3      hi for all the people who have seen this wonde...          1
4      I recently bought the DVD, forgetting just how...          0
...                                                  ...        ...
49995  OK, lets start with the best. the building. al...          0
49996  The British 'heritage film' industry is out of...          0
49997  I don't even know where to begin on this one. ...          0
49998  Richard Tyler is a little boy who is scared of...          0
49999  I waited long to watch this movie. Also becaus...          1

[50000 rows x 2 columns]


,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [8]:
### Defining the feature processing

TEXT = torchtext.legacy.data.Field(
    tokenize='spacy', # default splits on whitespace
    tokenizer_language='en_core_web_sm'
)

### Defining the label processing

LABEL = torchtext.legacy.data.LabelField(dtype=torch.long)

In [9]:
fields = [('TEXT_COLUMN_NAME', TEXT), ('LABEL_COLUMN_NAME', LABEL)]

dataset = torchtext.legacy.data.TabularDataset(
    path='movie_data.csv', format='csv',
    skip_header=True, fields=fields)

In [10]:
train_data, test_data = dataset.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 40000
Num Test: 10000


In [11]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(RANDOM_SEED))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')

Num Train: 34000
Num Validation: 6000


In [13]:
TEXT.build_vocab(train_data, max_size=VOCABULARY_SIZE)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [19]:
train_loader, valid_loader, test_loader = \
    torchtext.legacy.data.BucketIterator.splits(
        (train_data, valid_data, test_data),
         batch_size=BATCH_SIZE,
         sort_within_batch=False,
         sort_key=lambda x: len(x.TEXT_COLUMN_NAME),
         device=DEVICE
    )


In [20]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.TEXT_COLUMN_NAME.size()}')
    print(f'Target vector size: {batch.LABEL_COLUMN_NAME.size()}')
    break

Train
Text matrix size: torch.Size([1136, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([55, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([52, 128])
Target vector size: torch.Size([128])


In [21]:
class RNN(torch.nn.Module):
    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()

        self.embedding = torch.nn.Embedding(input_dim, embedding_dim)
        #self.rnn = torch.nn.RNN(embedding_dim,
        #                        hidden_dim,
        #                        nonlinearity='relu')
        self.rnn = torch.nn.LSTM(embedding_dim,
                                 hidden_dim)        
        
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        

    def forward(self, text):
        # text dim: [sentence length, batch size]
        
        embedded = self.embedding(text)
        # embedded dim: [sentence length, batch size, embedding dim]
        
        output, (hidden, cell) = self.rnn(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

In [22]:
torch.manual_seed(RANDOM_SEED)
model = RNN(input_dim=len(TEXT.vocab),
            embedding_dim=EMBEDDING_DIM,
            hidden_dim=HIDDEN_DIM,
            output_dim=NUM_CLASSES # could use 1 for binary classification
)

model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [23]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits = model(features)
            _, predicted_labels = torch.max(logits, 1)

            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

## Part 1: Training RNN With Varying Hyperparameters (Altering Hidden Layer Sizes Etc.)

## UGGGGHH Model below still doesn't converge.  Perhaps it is still too small for much knowledge to be encoded during training.  I will try increasing the EMBEDDING_DIM and HIDDEN_DIM below to see if it helps it converge.  (However there is a balance because if the network is too large then it takes an unreasonable amount of time to train!)

In [27]:
start_time = time.time()
NUM_EPOCHS = 3
BATCH_SIZE = 24
EMBEDDING_DIM = 8        #128
HIDDEN_DIM =  16         #256

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
               f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
               f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/003 | Batch 000/266 | Loss: 0.6880
Epoch: 001/003 | Batch 001/266 | Loss: 0.6881
Epoch: 001/003 | Batch 002/266 | Loss: 0.7042
Epoch: 001/003 | Batch 003/266 | Loss: 0.6912
Epoch: 001/003 | Batch 004/266 | Loss: 0.6937
Epoch: 001/003 | Batch 005/266 | Loss: 0.6992
Epoch: 001/003 | Batch 006/266 | Loss: 0.6954
Epoch: 001/003 | Batch 007/266 | Loss: 0.6914
Epoch: 001/003 | Batch 008/266 | Loss: 0.6864
Epoch: 001/003 | Batch 009/266 | Loss: 0.6901
Epoch: 001/003 | Batch 010/266 | Loss: 0.6900
Epoch: 001/003 | Batch 011/266 | Loss: 0.6962
Epoch: 001/003 | Batch 012/266 | Loss: 0.6867
Epoch: 001/003 | Batch 013/266 | Loss: 0.6911
Epoch: 001/003 | Batch 014/266 | Loss: 0.6932
Epoch: 001/003 | Batch 015/266 | Loss: 0.6898
Epoch: 001/003 | Batch 016/266 | Loss: 0.6990
Epoch: 001/003 | Batch 017/266 | Loss: 0.6911
Epoch: 001/003 | Batch 018/266 | Loss: 0.6919
Epoch: 001/003 | Batch 019/266 | Loss: 0.6957
Epoch: 001/003 | Batch 020/266 | Loss: 0.7032
Epoch: 001/003 | Batch 021/266 | L

Epoch: 001/003 | Batch 179/266 | Loss: 0.7011
Epoch: 001/003 | Batch 180/266 | Loss: 0.6967
Epoch: 001/003 | Batch 181/266 | Loss: 0.6877
Epoch: 001/003 | Batch 182/266 | Loss: 0.7068
Epoch: 001/003 | Batch 183/266 | Loss: 0.6887
Epoch: 001/003 | Batch 184/266 | Loss: 0.6880
Epoch: 001/003 | Batch 185/266 | Loss: 0.6877
Epoch: 001/003 | Batch 186/266 | Loss: 0.6967
Epoch: 001/003 | Batch 187/266 | Loss: 0.6883
Epoch: 001/003 | Batch 188/266 | Loss: 0.6957
Epoch: 001/003 | Batch 189/266 | Loss: 0.6866
Epoch: 001/003 | Batch 190/266 | Loss: 0.6889
Epoch: 001/003 | Batch 191/266 | Loss: 0.6889
Epoch: 001/003 | Batch 192/266 | Loss: 0.6875
Epoch: 001/003 | Batch 193/266 | Loss: 0.6986
Epoch: 001/003 | Batch 194/266 | Loss: 0.6899
Epoch: 001/003 | Batch 195/266 | Loss: 0.6889
Epoch: 001/003 | Batch 196/266 | Loss: 0.7052
Epoch: 001/003 | Batch 197/266 | Loss: 0.6865
Epoch: 001/003 | Batch 198/266 | Loss: 0.6924
Epoch: 001/003 | Batch 199/266 | Loss: 0.6993
Epoch: 001/003 | Batch 200/266 | L

Epoch: 002/003 | Batch 090/266 | Loss: 0.6898
Epoch: 002/003 | Batch 091/266 | Loss: 0.7024
Epoch: 002/003 | Batch 092/266 | Loss: 0.6900
Epoch: 002/003 | Batch 093/266 | Loss: 0.6890
Epoch: 002/003 | Batch 094/266 | Loss: 0.6898
Epoch: 002/003 | Batch 095/266 | Loss: 0.6907
Epoch: 002/003 | Batch 096/266 | Loss: 0.6876
Epoch: 002/003 | Batch 097/266 | Loss: 0.6915
Epoch: 002/003 | Batch 098/266 | Loss: 0.6881
Epoch: 002/003 | Batch 099/266 | Loss: 0.6975
Epoch: 002/003 | Batch 100/266 | Loss: 0.6884
Epoch: 002/003 | Batch 101/266 | Loss: 0.6895
Epoch: 002/003 | Batch 102/266 | Loss: 0.6888
Epoch: 002/003 | Batch 103/266 | Loss: 0.6898
Epoch: 002/003 | Batch 104/266 | Loss: 0.6887
Epoch: 002/003 | Batch 105/266 | Loss: 0.7025
Epoch: 002/003 | Batch 106/266 | Loss: 0.6902
Epoch: 002/003 | Batch 107/266 | Loss: 0.6908
Epoch: 002/003 | Batch 108/266 | Loss: 0.6885
Epoch: 002/003 | Batch 109/266 | Loss: 0.6887
Epoch: 002/003 | Batch 110/266 | Loss: 0.6879
Epoch: 002/003 | Batch 111/266 | L

Epoch: 003/003 | Batch 001/266 | Loss: 0.6920
Epoch: 003/003 | Batch 002/266 | Loss: 0.6870
Epoch: 003/003 | Batch 003/266 | Loss: 0.6879
Epoch: 003/003 | Batch 004/266 | Loss: 0.6874
Epoch: 003/003 | Batch 005/266 | Loss: 0.6932
Epoch: 003/003 | Batch 006/266 | Loss: 0.6890
Epoch: 003/003 | Batch 007/266 | Loss: 0.6902
Epoch: 003/003 | Batch 008/266 | Loss: 0.6918
Epoch: 003/003 | Batch 009/266 | Loss: 0.6904
Epoch: 003/003 | Batch 010/266 | Loss: 0.6890
Epoch: 003/003 | Batch 011/266 | Loss: 0.6895
Epoch: 003/003 | Batch 012/266 | Loss: 0.7047
Epoch: 003/003 | Batch 013/266 | Loss: 0.6938
Epoch: 003/003 | Batch 014/266 | Loss: 0.6902
Epoch: 003/003 | Batch 015/266 | Loss: 0.6882
Epoch: 003/003 | Batch 016/266 | Loss: 0.6878
Epoch: 003/003 | Batch 017/266 | Loss: 0.6868
Epoch: 003/003 | Batch 018/266 | Loss: 0.6928
Epoch: 003/003 | Batch 019/266 | Loss: 0.6876
Epoch: 003/003 | Batch 020/266 | Loss: 0.6903
Epoch: 003/003 | Batch 021/266 | Loss: 0.6880
Epoch: 003/003 | Batch 022/266 | L

Epoch: 003/003 | Batch 180/266 | Loss: 0.6891
Epoch: 003/003 | Batch 181/266 | Loss: 0.6926
Epoch: 003/003 | Batch 182/266 | Loss: 0.6893
Epoch: 003/003 | Batch 183/266 | Loss: 0.6885
Epoch: 003/003 | Batch 184/266 | Loss: 0.6912
Epoch: 003/003 | Batch 185/266 | Loss: 0.6898
Epoch: 003/003 | Batch 186/266 | Loss: 0.6884
Epoch: 003/003 | Batch 187/266 | Loss: 0.6909
Epoch: 003/003 | Batch 188/266 | Loss: 0.7082
Epoch: 003/003 | Batch 189/266 | Loss: 0.6891
Epoch: 003/003 | Batch 190/266 | Loss: 0.6882
Epoch: 003/003 | Batch 191/266 | Loss: 0.6884
Epoch: 003/003 | Batch 192/266 | Loss: 0.6882
Epoch: 003/003 | Batch 193/266 | Loss: 0.6882
Epoch: 003/003 | Batch 194/266 | Loss: 0.6884
Epoch: 003/003 | Batch 195/266 | Loss: 0.6884
Epoch: 003/003 | Batch 196/266 | Loss: 0.6883
Epoch: 003/003 | Batch 197/266 | Loss: 0.6893
Epoch: 003/003 | Batch 198/266 | Loss: 0.6880
Epoch: 003/003 | Batch 199/266 | Loss: 0.7042
Epoch: 003/003 | Batch 200/266 | Loss: 0.6883
Epoch: 003/003 | Batch 201/266 | L

## HOORAYYY THE MODEL BELOW CONVERGED REASONABLY!! THIS WAS THE BEST RNN MODEL I WAS ABLE TO GET BY VARYING THE HYPERPARAMETERS.  THUS THIS WILL BE THE MODEL I SAVE TO LOAD IN THE MAIN JUPYTER NOTEBOOK

In [33]:
start_time = time.time()
NUM_EPOCHS = 8
BATCH_SIZE = 24
EMBEDDING_DIM = 16        #128
HIDDEN_DIM =  24         #256
LEARNING_RATE = 0.02

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text = batch_data.TEXT_COLUMN_NAME.to(DEVICE)
        labels = batch_data.LABEL_COLUMN_NAME.to(DEVICE)

        ### FORWARD AND BACK PROP
        logits = model(text)
        loss = F.cross_entropy(logits, labels)
        optimizer.zero_grad()
        
        loss.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
               f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
               f'Loss: {loss:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/008 | Batch 000/266 | Loss: 0.6944
Epoch: 001/008 | Batch 001/266 | Loss: 0.6880
Epoch: 001/008 | Batch 002/266 | Loss: 0.6885
Epoch: 001/008 | Batch 003/266 | Loss: 0.6895
Epoch: 001/008 | Batch 004/266 | Loss: 0.6884
Epoch: 001/008 | Batch 005/266 | Loss: 0.6885
Epoch: 001/008 | Batch 006/266 | Loss: 0.6886
Epoch: 001/008 | Batch 007/266 | Loss: 0.6881
Epoch: 001/008 | Batch 008/266 | Loss: 0.6878
Epoch: 001/008 | Batch 009/266 | Loss: 0.6879
Epoch: 001/008 | Batch 010/266 | Loss: 0.6878
Epoch: 001/008 | Batch 011/266 | Loss: 0.6842
Epoch: 001/008 | Batch 012/266 | Loss: 0.7145
Epoch: 001/008 | Batch 013/266 | Loss: 0.6881
Epoch: 001/008 | Batch 014/266 | Loss: 0.6963
Epoch: 001/008 | Batch 015/266 | Loss: 0.6877
Epoch: 001/008 | Batch 016/266 | Loss: 0.6956
Epoch: 001/008 | Batch 017/266 | Loss: 0.7093
Epoch: 001/008 | Batch 018/266 | Loss: 0.6908
Epoch: 001/008 | Batch 019/266 | Loss: 0.6893
Epoch: 001/008 | Batch 020/266 | Loss: 0.6867
Epoch: 001/008 | Batch 021/266 | L

Epoch: 001/008 | Batch 179/266 | Loss: 0.6907
Epoch: 001/008 | Batch 180/266 | Loss: 0.6897
Epoch: 001/008 | Batch 181/266 | Loss: 0.6905
Epoch: 001/008 | Batch 182/266 | Loss: 0.6944
Epoch: 001/008 | Batch 183/266 | Loss: 0.6875
Epoch: 001/008 | Batch 184/266 | Loss: 0.6884
Epoch: 001/008 | Batch 185/266 | Loss: 0.6919
Epoch: 001/008 | Batch 186/266 | Loss: 0.6888
Epoch: 001/008 | Batch 187/266 | Loss: 0.6871
Epoch: 001/008 | Batch 188/266 | Loss: 0.6887
Epoch: 001/008 | Batch 189/266 | Loss: 0.6875
Epoch: 001/008 | Batch 190/266 | Loss: 0.6873
Epoch: 001/008 | Batch 191/266 | Loss: 0.6847
Epoch: 001/008 | Batch 192/266 | Loss: 0.6877
Epoch: 001/008 | Batch 193/266 | Loss: 0.6831
Epoch: 001/008 | Batch 194/266 | Loss: 0.6876
Epoch: 001/008 | Batch 195/266 | Loss: 0.6901
Epoch: 001/008 | Batch 196/266 | Loss: 0.6918
Epoch: 001/008 | Batch 197/266 | Loss: 0.6847
Epoch: 001/008 | Batch 198/266 | Loss: 0.6925
Epoch: 001/008 | Batch 199/266 | Loss: 0.6905
Epoch: 001/008 | Batch 200/266 | L

Epoch: 002/008 | Batch 090/266 | Loss: 0.6882
Epoch: 002/008 | Batch 091/266 | Loss: 0.6877
Epoch: 002/008 | Batch 092/266 | Loss: 0.6872
Epoch: 002/008 | Batch 093/266 | Loss: 0.6947
Epoch: 002/008 | Batch 094/266 | Loss: 0.6915
Epoch: 002/008 | Batch 095/266 | Loss: 0.6920
Epoch: 002/008 | Batch 096/266 | Loss: 0.6860
Epoch: 002/008 | Batch 097/266 | Loss: 0.6889
Epoch: 002/008 | Batch 098/266 | Loss: 0.6867
Epoch: 002/008 | Batch 099/266 | Loss: 0.6863
Epoch: 002/008 | Batch 100/266 | Loss: 0.6870
Epoch: 002/008 | Batch 101/266 | Loss: 0.6909
Epoch: 002/008 | Batch 102/266 | Loss: 0.6853
Epoch: 002/008 | Batch 103/266 | Loss: 0.6877
Epoch: 002/008 | Batch 104/266 | Loss: 0.6927
Epoch: 002/008 | Batch 105/266 | Loss: 0.6912
Epoch: 002/008 | Batch 106/266 | Loss: 0.6941
Epoch: 002/008 | Batch 107/266 | Loss: 0.6903
Epoch: 002/008 | Batch 108/266 | Loss: 0.6896
Epoch: 002/008 | Batch 109/266 | Loss: 0.6886
Epoch: 002/008 | Batch 110/266 | Loss: 0.6896
Epoch: 002/008 | Batch 111/266 | L

Epoch: 003/008 | Batch 001/266 | Loss: 0.6888
Epoch: 003/008 | Batch 002/266 | Loss: 0.6869
Epoch: 003/008 | Batch 003/266 | Loss: 0.6979
Epoch: 003/008 | Batch 004/266 | Loss: 0.6883
Epoch: 003/008 | Batch 005/266 | Loss: 0.6880
Epoch: 003/008 | Batch 006/266 | Loss: 0.6860
Epoch: 003/008 | Batch 007/266 | Loss: 0.6879
Epoch: 003/008 | Batch 008/266 | Loss: 0.6893
Epoch: 003/008 | Batch 009/266 | Loss: 0.6869
Epoch: 003/008 | Batch 010/266 | Loss: 0.6925
Epoch: 003/008 | Batch 011/266 | Loss: 0.6917
Epoch: 003/008 | Batch 012/266 | Loss: 0.6866
Epoch: 003/008 | Batch 013/266 | Loss: 0.6869
Epoch: 003/008 | Batch 014/266 | Loss: 0.6887
Epoch: 003/008 | Batch 015/266 | Loss: 0.6879
Epoch: 003/008 | Batch 016/266 | Loss: 0.6877
Epoch: 003/008 | Batch 017/266 | Loss: 0.6900
Epoch: 003/008 | Batch 018/266 | Loss: 0.6873
Epoch: 003/008 | Batch 019/266 | Loss: 0.6875
Epoch: 003/008 | Batch 020/266 | Loss: 0.6870
Epoch: 003/008 | Batch 021/266 | Loss: 0.6875
Epoch: 003/008 | Batch 022/266 | L

Epoch: 003/008 | Batch 180/266 | Loss: 0.4722
Epoch: 003/008 | Batch 181/266 | Loss: 0.5038
Epoch: 003/008 | Batch 182/266 | Loss: 0.5696
Epoch: 003/008 | Batch 183/266 | Loss: 0.4981
Epoch: 003/008 | Batch 184/266 | Loss: 0.3751
Epoch: 003/008 | Batch 185/266 | Loss: 0.4263
Epoch: 003/008 | Batch 186/266 | Loss: 0.4425
Epoch: 003/008 | Batch 187/266 | Loss: 0.4750
Epoch: 003/008 | Batch 188/266 | Loss: 0.5335
Epoch: 003/008 | Batch 189/266 | Loss: 0.4419
Epoch: 003/008 | Batch 190/266 | Loss: 0.4706
Epoch: 003/008 | Batch 191/266 | Loss: 0.4373
Epoch: 003/008 | Batch 192/266 | Loss: 0.3908
Epoch: 003/008 | Batch 193/266 | Loss: 0.4861
Epoch: 003/008 | Batch 194/266 | Loss: 0.5015
Epoch: 003/008 | Batch 195/266 | Loss: 0.3710
Epoch: 003/008 | Batch 196/266 | Loss: 0.4185
Epoch: 003/008 | Batch 197/266 | Loss: 0.4018
Epoch: 003/008 | Batch 198/266 | Loss: 0.3985
Epoch: 003/008 | Batch 199/266 | Loss: 0.4885
Epoch: 003/008 | Batch 200/266 | Loss: 0.4255
Epoch: 003/008 | Batch 201/266 | L

...

Epoch: 005/008 | Batch 181/266 | Loss: 0.2872
Epoch: 005/008 | Batch 182/266 | Loss: 0.1772
Epoch: 005/008 | Batch 183/266 | Loss: 0.1159
Epoch: 005/008 | Batch 184/266 | Loss: 0.1803
Epoch: 005/008 | Batch 185/266 | Loss: 0.2297
Epoch: 005/008 | Batch 186/266 | Loss: 0.2495
Epoch: 005/008 | Batch 187/266 | Loss: 0.1687
Epoch: 005/008 | Batch 188/266 | Loss: 0.1942
Epoch: 005/008 | Batch 189/266 | Loss: 0.1569
Epoch: 005/008 | Batch 190/266 | Loss: 0.2514
Epoch: 005/008 | Batch 191/266 | Loss: 0.2546
Epoch: 005/008 | Batch 192/266 | Loss: 0.2227
Epoch: 005/008 | Batch 193/266 | Loss: 0.2774
Epoch: 005/008 | Batch 194/266 | Loss: 0.2359
Epoch: 005/008 | Batch 195/266 | Loss: 0.2058
Epoch: 005/008 | Batch 196/266 | Loss: 0.2813
Epoch: 005/008 | Batch 197/266 | Loss: 0.2633
Epoch: 005/008 | Batch 198/266 | Loss: 0.2230
Epoch: 005/008 | Batch 199/266 | Loss: 0.2015
Epoch: 005/008 | Batch 200/266 | Loss: 0.1906
Epoch: 005/008 | Batch 201/266 | Loss: 0.1732
Epoch: 005/008 | Batch 202/266 | L

Epoch: 006/008 | Batch 092/266 | Loss: 0.1889
Epoch: 006/008 | Batch 093/266 | Loss: 0.1378
Epoch: 006/008 | Batch 094/266 | Loss: 0.1296
Epoch: 006/008 | Batch 095/266 | Loss: 0.1110
Epoch: 006/008 | Batch 096/266 | Loss: 0.1823
Epoch: 006/008 | Batch 097/266 | Loss: 0.1198
Epoch: 006/008 | Batch 098/266 | Loss: 0.1585
Epoch: 006/008 | Batch 099/266 | Loss: 0.1443
Epoch: 006/008 | Batch 100/266 | Loss: 0.0780
Epoch: 006/008 | Batch 101/266 | Loss: 0.1148
Epoch: 006/008 | Batch 102/266 | Loss: 0.1645
Epoch: 006/008 | Batch 103/266 | Loss: 0.1145
Epoch: 006/008 | Batch 104/266 | Loss: 0.0934
Epoch: 006/008 | Batch 105/266 | Loss: 0.1002
Epoch: 006/008 | Batch 106/266 | Loss: 0.1959
Epoch: 006/008 | Batch 107/266 | Loss: 0.1667
Epoch: 006/008 | Batch 108/266 | Loss: 0.0738
Epoch: 006/008 | Batch 109/266 | Loss: 0.1472
Epoch: 006/008 | Batch 110/266 | Loss: 0.1215
Epoch: 006/008 | Batch 111/266 | Loss: 0.0614
Epoch: 006/008 | Batch 112/266 | Loss: 0.1199
Epoch: 006/008 | Batch 113/266 | L

Epoch: 007/008 | Batch 003/266 | Loss: 0.0639
Epoch: 007/008 | Batch 004/266 | Loss: 0.1504
Epoch: 007/008 | Batch 005/266 | Loss: 0.1172
Epoch: 007/008 | Batch 006/266 | Loss: 0.0720
Epoch: 007/008 | Batch 007/266 | Loss: 0.0579
Epoch: 007/008 | Batch 008/266 | Loss: 0.0515
Epoch: 007/008 | Batch 009/266 | Loss: 0.0838
Epoch: 007/008 | Batch 010/266 | Loss: 0.1853
Epoch: 007/008 | Batch 011/266 | Loss: 0.1345
Epoch: 007/008 | Batch 012/266 | Loss: 0.0373
Epoch: 007/008 | Batch 013/266 | Loss: 0.0568
Epoch: 007/008 | Batch 014/266 | Loss: 0.0487
Epoch: 007/008 | Batch 015/266 | Loss: 0.1226
Epoch: 007/008 | Batch 016/266 | Loss: 0.0895
Epoch: 007/008 | Batch 017/266 | Loss: 0.0995
Epoch: 007/008 | Batch 018/266 | Loss: 0.0646
Epoch: 007/008 | Batch 019/266 | Loss: 0.0764
Epoch: 007/008 | Batch 020/266 | Loss: 0.2157
Epoch: 007/008 | Batch 021/266 | Loss: 0.0981
Epoch: 007/008 | Batch 022/266 | Loss: 0.0389
Epoch: 007/008 | Batch 023/266 | Loss: 0.1553
Epoch: 007/008 | Batch 024/266 | L

Epoch: 007/008 | Batch 182/266 | Loss: 0.0636
Epoch: 007/008 | Batch 183/266 | Loss: 0.1222
Epoch: 007/008 | Batch 184/266 | Loss: 0.0462
Epoch: 007/008 | Batch 185/266 | Loss: 0.0586
Epoch: 007/008 | Batch 186/266 | Loss: 0.0672
Epoch: 007/008 | Batch 187/266 | Loss: 0.0596
Epoch: 007/008 | Batch 188/266 | Loss: 0.1320
Epoch: 007/008 | Batch 189/266 | Loss: 0.1222
Epoch: 007/008 | Batch 190/266 | Loss: 0.1490
Epoch: 007/008 | Batch 191/266 | Loss: 0.0819
Epoch: 007/008 | Batch 192/266 | Loss: 0.1482
Epoch: 007/008 | Batch 193/266 | Loss: 0.0828
Epoch: 007/008 | Batch 194/266 | Loss: 0.0356
Epoch: 007/008 | Batch 195/266 | Loss: 0.1933
Epoch: 007/008 | Batch 196/266 | Loss: 0.0940
Epoch: 007/008 | Batch 197/266 | Loss: 0.0460
Epoch: 007/008 | Batch 198/266 | Loss: 0.2468
Epoch: 007/008 | Batch 199/266 | Loss: 0.1178
Epoch: 007/008 | Batch 200/266 | Loss: 0.1225
Epoch: 007/008 | Batch 201/266 | Loss: 0.1082
Epoch: 007/008 | Batch 202/266 | Loss: 0.0853
Epoch: 007/008 | Batch 203/266 | L

Epoch: 008/008 | Batch 093/266 | Loss: 0.0883
Epoch: 008/008 | Batch 094/266 | Loss: 0.1161
Epoch: 008/008 | Batch 095/266 | Loss: 0.1395
Epoch: 008/008 | Batch 096/266 | Loss: 0.0896
Epoch: 008/008 | Batch 097/266 | Loss: 0.1375
Epoch: 008/008 | Batch 098/266 | Loss: 0.0451
Epoch: 008/008 | Batch 099/266 | Loss: 0.0089
Epoch: 008/008 | Batch 100/266 | Loss: 0.0714
Epoch: 008/008 | Batch 101/266 | Loss: 0.0900
Epoch: 008/008 | Batch 102/266 | Loss: 0.0326
Epoch: 008/008 | Batch 103/266 | Loss: 0.0148
Epoch: 008/008 | Batch 104/266 | Loss: 0.0988
Epoch: 008/008 | Batch 105/266 | Loss: 0.0458
Epoch: 008/008 | Batch 106/266 | Loss: 0.0727
Epoch: 008/008 | Batch 107/266 | Loss: 0.0267
Epoch: 008/008 | Batch 108/266 | Loss: 0.0901
Epoch: 008/008 | Batch 109/266 | Loss: 0.0452
Epoch: 008/008 | Batch 110/266 | Loss: 0.0513
Epoch: 008/008 | Batch 111/266 | Loss: 0.0802
Epoch: 008/008 | Batch 112/266 | Loss: 0.0765
Epoch: 008/008 | Batch 113/266 | Loss: 0.0576
Epoch: 008/008 | Batch 114/266 | L

## Below I print out the final weights of the trained RNN Model before saving them to a file

In [49]:
print(model.state_dict())

OrderedDict([('embedding.weight', tensor([[-0.4851, -0.2355,  0.0630,  ...,  1.3268,  0.0636,  0.6657],
        [-0.9825, -1.8656, -1.0225,  ...,  1.0825, -0.7484, -3.3752],
        [-1.4598,  0.1361, -3.5899,  ...,  0.6832,  0.1474, -1.1503],
        ...,
        [ 0.1470,  0.8990, -0.5071,  ..., -0.6089, -0.7529,  0.5489],
        [-0.7083, -0.8101,  0.9433,  ...,  1.0532, -0.9461, -0.5724],
        [ 0.0263, -1.8620,  1.7259,  ...,  0.4474, -0.9539, -0.4021]])), ('rnn.weight_ih_l0', tensor([[-0.1460, -0.5099, -0.3953,  ...,  0.2937, -0.0484,  0.7155],
        [ 0.7581,  0.3694, -0.4216,  ..., -0.5087,  0.0948,  1.0980],
        [-0.8754, -0.6928,  0.4072,  ...,  0.4331,  0.5352, -0.1056],
        ...,
        [-0.2470,  0.9151,  0.5359,  ...,  0.0547,  0.9964,  0.4931],
        [-1.0646,  0.4829,  0.1846,  ...,  0.5054, -0.1292,  0.6290],
        [ 1.1518, -0.8127, -0.1810,  ...,  0.0412, -0.4375,  0.7093]])), ('rnn.weight_hh_l0', tensor([[-0.2916, -0.3677, -0.2623,  ...,  0.1071,  

## I learned that below is the way to save a model (I am just saving it to my current directory under the name bestRNNweights).  I can then load these weights into a model with the same dimensions in my other jupyter notebook and I won't have to retrain the model!

In [52]:
torch.save(model.state_dict(), "./bestRNNweights")

In [53]:
print(model)

RNN(
  (embedding): Embedding(20002, 16)
  (rnn): LSTM(16, 24)
  (fc): Linear(in_features=24, out_features=2, bias=True)
)
